## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import folium

## Data Read-in

In [2]:
df = pd.read_csv('redfin_2023-07-05-06-23-26.csv')

## Data Clean

In [3]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [4]:
df = df.dropna(subset=['SOLD DATE'])

In [5]:
# Define list of desired months (excluding current month)
desired_months = ['June']

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [6]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

PRICE
False    968
Name: count, dtype: int64
-------
$/SQUARE FEET
False    965
True       3
Name: count, dtype: int64
-------
YEAR BUILT
False    968
Name: count, dtype: int64
-------


In [7]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE


In [8]:
df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [9]:
df_filtered.sort_values(by='PRICE',ascending=True).head(20)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
625,PAST SALE,June-6-2023,Condo/Co-op,635 83rd St #48,Miami Beach,FL,33141.0,150.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/635-83rd...,MARMLS,A11332007,N,Y,25.868651,-80.125829
819,PAST SALE,June-30-2023,Condo/Co-op,2780 NE 183rd St #508,Aventura,FL,33160.0,259.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/2780-NE-183...,MARMLS,A11314553,N,Y,25.944585,-80.144209
856,PAST SALE,June-28-2023,Condo/Co-op,19195 NE 36th Ct #702,Aventura,FL,33180.0,600.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/19195-Mysti...,MARMLS,A11319356,N,Y,25.953453,-80.126851
382,PAST SALE,June-12-2023,Condo/Co-op,2555 Collins Ave Ph 107,Miami Beach,FL,33140.0,710.0,1.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/2555-Col...,MARMLS,A11375016,N,Y,25.801681,-80.125790
689,PAST SALE,June-29-2023,Condo/Co-op,1355 NE 167th St #202,North Miami Beach,FL,33162.0,72500.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/13...,Beaches MLS,F10370034,N,Y,25.929738,-80.173163
159,PAST SALE,June-2-2023,Condo/Co-op,52 NE 204th St #31,Miami Gardens,FL,33179.0,100000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Gardens/52-NE-...,MARMLS,A11379000,N,Y,25.962917,-80.200185
622,PAST SALE,June-30-2023,Condo/Co-op,1750 NE 191st St Unit 614-2,Miami,FL,33179.0,100000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/17...,MARMLS,A11359284,N,Y,25.949213,-80.167412
629,PAST SALE,June-16-2023,Condo/Co-op,20337 NE 2nd Ave #4,Miami,FL,33179.0,105000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/20337-NE-2nd-A...,MARMLS,A11357722,N,Y,25.962503,-80.197837
121,PAST SALE,June-9-2023,Condo/Co-op,15610 NE 6th Ave Unit 36D,Miami,FL,33162.0,107000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/15610-NE-6th-A...,Beaches MLS,F10379966,N,Y,25.918155,-80.188500
825,PAST SALE,June-5-2023,Condo/Co-op,13500 NE 3rd Ct #206,North Miami,FL,33161.0,125000.0,1.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami/13500-NE...,MARMLS,A11308729,N,Y,25.899624,-80.192301


In [20]:
print(df_filtered['URL'].iloc[382])

https://www.redfin.com/FL/Miami-Beach/2555-Collins-Ave-33140/unit-107/home/42786770


In [15]:
# Correct the prices, if needed
df_filtered.at[625,'PRICE']=(150000)
df_filtered.at[819,'PRICE']=(259000)
df_filtered.at[856,'PRICE']=(600000)
df_filtered.at[382,'PRICE']=(710000)

In [16]:
# Find problem psf by searching for a '0' value
df_filtered.loc[df_filtered['$/SQUARE FEET'] == '0'][['SOLD DATE','ADDRESS','CITY','$/SQUARE FEET','PRICE','SQUARE FEET']]

,SOLD DATE,ADDRESS,CITY,$/SQUARE FEET,PRICE,SQUARE FEET


In [21]:
# # Corrections, if needed
df_filtered.at[625,'$/SQUARE FEET']=(150000/577)
df_filtered.at[819,'$/SQUARE FEET']=(259000/887)
df_filtered.at[856,'$/SQUARE FEET']=(600000/1356)
df_filtered.at[382,'$/SQUARE FEET']=(710000/861)

In [22]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['PRICE','ADDRESS','CITY','$/SQUARE FEET']]

,PRICE,ADDRESS,CITY,$/SQUARE FEET
689,72500.0,1355 NE 167th St #202,North Miami Beach,82.0
622,100000.0,1750 NE 191st St Unit 614-2,Miami,97.0
284,230000.0,5300 NW 87th Ave #613,Doral,111.0
474,220000.0,20850 San Simeon Way #106,Miami,139.0
159,100000.0,52 NE 204th St #31,Miami Gardens,141.0
420,195000.0,445 NE 195th St #327,Miami,142.0
89,165500.0,1351 NE Miami Gardens Dr Unit 325E,Miami,145.0
596,125000.0,1351 NE Miami Gardens Dr Unit 723E,Miami,145.0
258,215000.0,800 NE 195th St #206,Miami,146.0
7,147000.0,470 NE 18th Ave #107,Homestead,146.0


In [23]:
print(df_filtered.URL.iloc[689])

https://www.redfin.com/FL/North-Miami-Beach/1355-NE-167th-St-33162/unit-202/home/43006115


In [17]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

## Make Maps

In [24]:
### Create a price column formatted as currency ###
df_filtered['PRICE_AS_CURRENCY'] = df_filtered['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_filtered['YEAR BUILT DISPLAY'] = df_filtered['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_filtered['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_filtered['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [25]:
df_filtered = df_filtered.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_filtered['COLOR'] = ''
### Create RANK column ###
df_filtered['RANK'] = 0
### Insert RANK values ###
df_filtered['RANK'] = range(1, len(df_filtered) + 1)
# use numpy to assign values to the 'COLOR' column
df_filtered['COLOR'] = np.where(df_filtered['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [26]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [27]:
### Drop the columns ###
df_filtered = df_filtered.drop(columns=columns_drop)

In [28]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT DISPLAY']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [29]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"June 2023 Condo Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

for index, row in df_filtered.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map

In [30]:
m.save('index.html')

## Summary Info

In [31]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [32]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK'],
      dtype='object')

In [33]:
df_filtered['FULL_ADDRESS'] = df_filtered['ADDRESS'] + ' ' + df_filtered['CITY']

In [34]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['URL'])

https://www.redfin.com/FL/Miami-Beach/834-Ocean-Dr-33139/unit-506/home/42795170


In [35]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")

Most Expensive
FENDI CHATEAU, 9349 Collins Ave #406 Surfside | Price $19,750,000 | $3,453 psf | Year built: 2016
Least Expensive
Mar-len Gardens, 1355 NE 167th St #202 North Miami Beach | Price $72,500 | $82 psf | Year built: 1964
Highest Price Per Square Foot
FENDI CHATEAU, 9349 Collins Ave #406 Surfside | Price $19,750,000 | $3,453 psf | Year built: 2016
Lowest Price Per Square Foot
Mar-len Gardens, 1355 NE 167th St #202 North Miami Beach | Price $72,500 | $82 psf | Year built: 1964
Newest
VILLA VALENCIA, 515 Valencia #903 Coral Gables | Price $3,700,000 | $1,181 psf | Year built: 2022
Oldest
CASA GRANDE CONDO, 834 Ocean Dr #506 Miami Beach | Price $400,000 | $702 psf | Year built: 1923


## Time on Market Calculator

In [29]:
# print(df_Current.loc[df_Current['YEAR BUILT'].idxmin()])

In [48]:
# from datetime import datetime, timedelta

# date1 = datetime(2023, 4, 10) ## List (Earlier) date
# date2 = datetime(2023, 5, 17) ## Close (Later) date

# delta = date2 - date1
# num_days = delta.days

# print(num_days)

37


## Map URL Snagger

In [36]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [37]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MD_condo_sales_month_ending_june_2023


## Get Summary Data

In [38]:
print('SALES INFO')
print(f'Number of sales: {len(df_filtered)}')
print('--------')
print(f'Total sale price: ${df_filtered["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_filtered["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_filtered["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_filtered["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_filtered["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_filtered["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_filtered["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_filtered["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_filtered["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_filtered["YEAR BUILT"].mean()}')

SALES INFO
Number of sales: 968
--------
Total sale price: $699,661,349
--------
Median sale price: $400,000
--------
Max sale price: $19,750,000
--------
Min sale price: $72,500
------------------------------------------------
PSF INFO
Max price per square foot: $3,453
--------
Min price per square foot: $82
--------
Median price per square foot: $433
------------------------------------------------
CONDO AGES
Newest building: 2022.0
----------
Oldest building: 1923.0
----------
Average building age: 1988.3543388429753
